In [1]:
!pip install transformers

In [3]:
!pip install -q kaggle

In [160]:
from google.colab import files
files.upload()

{}

In [5]:
!mkdir ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets list

ref                                                            title                                               size  lastUpdated          downloadCount  
-------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
arashnic/hr-analytics-job-change-of-data-scientists            HR Analytics: Job Change of Data Scientists        295KB  2020-12-07 00:25:10           2027  
utkarshxy/who-worldhealth-statistics-2020-complete             World Health 2020 🌏 | For Geospatial Analysis        1MB  2021-01-01 15:16:17             77  
babyoda/access-to-computers-from-home-oecd                     Access to Computers From Home OECD                   3KB  2020-12-04 10:50:09            192  
shashwatwork/impact-of-covid19-pandemic-on-the-global-economy  Impact of Covid-19 Pandemic on the Global Economy    1MB  2020-11-29 14:16:30           1138  
emmanuelleai/world-marathons-majors                 

In [9]:
!kaggle datasets download -d narendrageek/mental-health-faq-for-chatbot

  0% 0.00/50.1k [00:00<?, ?B/s]
100% 50.1k/50.1k [00:00<00:00, 23.9MB/s]


In [10]:
!mkdir faq_data
!unzip mental-health-faq-for-chatbot.zip -d faq_data

Archive:  mental-health-faq-for-chatbot.zip
  inflating: faq_data/Mental_Health_FAQ.csv  


In [11]:
!mkdir faq_data_2
!kaggle datasets download -d jeromeblanchet/hotpotqa-question-answering-dataset

 97% 234M/241M [00:02<00:00, 58.4MB/s]
100% 241M/241M [00:02<00:00, 107MB/s] 


In [12]:
!unzip hotpotqa-question-answering-dataset.zip -d faq_data_2

Archive:  hotpotqa-question-answering-dataset.zip
  inflating: faq_data_2/hotpot_dev_distractor_v1.json  
  inflating: faq_data_2/hotpot_dev_fullwiki_v1.json  
  inflating: faq_data_2/hotpot_test_fullwiki_v1.json  
  inflating: faq_data_2/hotpot_train_v1.1.json  


In [13]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 782.7 MB 1.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [14]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import BertTokenizer
from transformers import TFBertForQuestionAnswering
from transformers import  TFTrainer, TFTrainingArguments
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import spacy
import pandas as pd
import numpy as np
import pandas as pd

#model_name = "bert-large-uncased"
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

In [15]:
data = pd.read_csv('./faq_data/Mental_Health_FAQ.csv')
data.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [16]:
print(data.shape)
print(data.isna().sum())

(98, 3)
Question_ID    0
Questions      0
Answers        0
dtype: int64


In [21]:
nlp = spacy.load('en_core_web_lg')

In [22]:
question_vectors = [nlp(x).vector for x in data['Questions'].values]

In [23]:
svd = TruncatedSVD(n_components=3)
svd_questions = svd.fit_transform(question_vectors)
cos_sim=cosine_similarity(svd_questions,svd_questions)


In [24]:
pd.DataFrame(cos_sim)[7].sort_values(ascending=False)[:10]


7     1.000000
47    0.999493
55    0.999171
17    0.996765
48    0.994752
37    0.994380
22    0.991148
23    0.990372
6     0.987505
49    0.986956
Name: 7, dtype: float32

In [25]:
my_sentece_vec = np.stack([nlp("Where can I find help?.").vector])
sentence_vector = np.append(question_vectors,my_sentece_vec,axis=0)

svd_questions = svd.fit_transform(sentence_vector)
cos_sim=cosine_similarity(svd_questions,svd_questions)

In [146]:
similar_questions = pd.DataFrame(cos_sim)[98].sort_values(ascending=False)[:10]
similar_questions

98    1.000000
19    0.999143
12    0.998734
33    0.998272
15    0.998008
46    0.997775
16    0.997639
25    0.997001
95    0.996550
8     0.995404
Name: 98, dtype: float32

In [147]:
print("Where can I find help?",'\n')
print(data['Questions'][19])

Question = "Where can I find help?"
Answer = data.iloc[similar_questions[2:3].index[0],2]

Where can I find help? 

Where can I go to find a support group?


In [148]:
data.iloc[similar_questions[1:2].index[0],2]

'Many people find peer support a helpful tool that can aid in their recovery. There are a variety of organizations that offer support groups for consumers, their family members and friends. Some support groups are peer-led, while others may be led by a mental health professional.'

In [149]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [150]:
input_dict=(tokenizer((Question,Answer),return_tensors='tf',padding=True, truncation=True ))

In [151]:
input_dict

{'input_ids': <tf.Tensor: shape=(2, 184), dtype=int32, numpy=
array([[  101,  2073,  2064,  1045,  2424,  2393,  1029,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0, 

In [152]:
model = None
model = TFBertForQuestionAnswering.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [153]:
outputs = model(input_dict)

In [158]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits

all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[1])
answer = ' '.join(all_tokens[tf.math.argmax(start_logits,1)[1] : tf.math.argmax(end_logits,1)[1]+1])

In [159]:
answer

'best place to start is by talking with someone you trust about your concerns , such as a family member , friend , clergy , healthcare provider , or other professionals'